In [15]:
from ultralytics import YOLO
import torch
import os

# Load a pre-trained model and customize it for your specific dataset
processDamageModel = YOLO('yolov8n.pt')  # Using a YOLOv8 nano model for simplicity; change as needed

# Start training
processDamageModel.train(
    data='./data.yaml',
    epochs=3,
    imgsz=640  # Adjust based on your specific needs
)

Ultralytics YOLOv8.2.76 🚀 Python-3.9.18 torch-2.4.0+cu121 CPU (Intel Xeon E5-2673 v4 2.30GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train39, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /mnt/batch/tasks/shared/LS_root/mounts/clusters/xusenshi1/code/Users/xusenshi/train/labels.cache... 68 images, 0 backgrounds, 0 corrupt: 100%|██████████| 68/68 [00:00<?, ?it/s]
/anaconda/envs/jupyter_env/lib/python3.9/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
val: Scanning /mnt/batch/tasks/shared/LS_root/mounts/clusters/xusenshi1/code/Users/xusenshi/valid/labels.cache... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]
        1/3         0G       3.15      4.603      2.366         21        640: 100%|██████████| 5/5 [01:00<00:00, 12.12s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:06<00:00,  6.40s/it]
        2/3         0G      2.913      4.327      2.084         84        640: 100%|██████████| 5/5 [00:58<00:00, 11.65s/it]
                 Class     Images  Instances      Box(P          R     

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f58a032d460>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [5]:
import cv2

# Open a connection to your webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Set the frame width and height (optional)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Loop to continuously get frames from the camera and run inference
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Run YOLOv8 inference on the frame
    results = model(frame)
    
    # Convert YOLOv8 predictions to a format that OpenCV can display
    annotated_frame = results[0].plot()

    # Display the resulting frame
    cv2.imshow('YOLOv8 Detection', annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Error: Could not open camera.
Failed to grab frame


[ WARN:0@977.280] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@977.284] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [14]:
from azureml.core import Workspace, Model
import os

model_path = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/xusenshi1/code/Users/xusenshi/processDamage.pt'  # Replace with your actual path
processDamageModel.save(model_path)

# Connect to Azure ML workspace
ws = Workspace.from_config()

# Register the model
model = Model.register(workspace=ws,
                       model_name='processDamageModel',  # Give your model a name
                       model_path=model_path)  # Path to the model file

AttributeError: 'Model' object has no attribute 'save'

In [10]:
%pwd

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/xusenshi1/code/Users/xusenshi'

In [ ]:
#Create the socre.py

import json
import torch
from ultralytics import YOLO

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'processDamage.pt')
    model = YOLO(model_path)

def run(data):
    try:
        # Parse input data
        input_data = json.loads(data)
        # Perform inference
        results = model(input_data['image'])
        # Return results
        return json.dumps({"predictions": results})
    except Exception as e:
        return json.dumps({"error": str(e)})

In [ ]:
#Create an environment

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

# Create an Azure ML environment
env = Environment(name="yolov8-env")

# Specify dependencies such as Python packages
env.python.conda_dependencies.add_pip_package("torch")
env.python.conda_dependencies.add_pip_package("ultralytics")

# Define the inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=env)


In [ ]:
#Deploy the model

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

# Define deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model
service = Model.deploy(workspace=ws,
                       name='process-damage-service',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

print(f"Scoring URI: {service.scoring_uri}")


In [ ]:
#Test the model

import requests
import json

# URL for the web service
scoring_uri = service.scoring_uri

# Input data
input_data = {
    "image": "base64-encoded-image-string-or-path-to-image"
}

# Make the request
response = requests.post(scoring_uri, data=json.dumps(input_data), headers={"Content-Type": "application/json"})

print(response.json())
